In [1]:
import glob
import os
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
import numpy as np
from sklearn.utils import shuffle

In [2]:
class_labels = {0: "Not Gunshot", 1: "Gunshot"}

def get_image_data(image_path):
    im = img_to_array(load_img(image_path).resize((256, 256), Image.LANCZOS))
    im = im / 255.0
    return im

def data(path, label):
    X = []
    Y = []
    image_paths = []
    files = glob.glob(path + '/*.png')
    for file in files:
        image = get_image_data(file)
        X.append(image)
        Y.append(label)
        image_paths.append(file)
    return X, Y, image_paths

In [3]:
img_width, img_height = 256, 256

# Data directories:
train_data_dir_gun = r'C:\Users\Personal\Documents\projects\KICS Second Project\convolution_nn\dataset'
validation_data_dir_gun = r'C:\Users\Personal\Documents\projects\KICS Second Project\convolution_nn\random valid'

In [4]:
epochs = 120
batch_size = 40

train_x1, train_y1, path_train1 = data(train_data_dir_gun, 1)
train_x = train_x1
train_y = train_y1
train_x, train_y = shuffle(train_x, train_y)
train_x = np.array(train_x)
train_y = np.array(train_y)
train_y = to_categorical(train_y, 2)

val_x1, val_y1, path_val_1 = data(validation_data_dir_gun, 1)
val_x = val_x1
val_y = val_y1
val_x, val_y = shuffle(val_x, val_y)
val_x = np.array(val_x)
val_y = np.array(val_y)
val_y = to_categorical(val_y, 2)

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(256, 256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\Users\Personal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=(val_x, val_y))

# Test prediction:
gunshot = []
true_gun = []

for im in val_x1:
    im = np.array(im)
    im = np.reshape(im, newshape=(1, 256, 256, 3))
    prediction = model.predict(im)
    classes_x = np.argmax(prediction, axis=1)

    # Get the class label from the dictionary
    predicted_class = class_labels[classes_x[0]]

    gunshot.append(predicted_class)
    true_gun.append("Gunshot")  # Assuming all validation images are gunshot
    print("True class", "Gunshot", " predicted class", predicted_class)

Epoch 1/120
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.8784 - loss: 0.2327 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/120
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/120
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/120
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/120
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/120
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/120
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/120
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 1.0000 - los

In [8]:
final_arrays = [path_val_1, true_gun, gunshot]

with open(r'C:\Users\Personal\Documents\projects\KICS Second Project\convolution_nn\new_12.csv', "a") as f:
    for i in range(len(final_arrays[0])):
        f.write(f"{final_arrays[0][i]},{final_arrays[1][i]},{final_arrays[2][i]}\n")